# Диагностика модели: тесты на серийную корреляцию

In [ ]:
import pandas as pd
import numpy as np

import statsmodels.formula.api as smf
import statsmodels.stats.api as sms # тесты
from statsmodels.iolib.summary2 import summary_col # вывод подгонки

from scipy.stats import chi2 # chi2-распределение и критические значения

# настройки визуализация
import pandas_datareader.data as web

import matplotlib.pyplot as plt

# Не показывать Warning
import warnings
warnings.simplefilter(action='ignore', category=Warning)

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

<i><b><span style="color: purple">Диагностика модели: тесты на гетероскедастичность</span></i></b>
<br>

Рассмотрим линейную регрессию 

$$
\begin{aligned}
	y_i&=x_i^\top\beta+u_i & i&=1,\ldots,n
\end{aligned}
$$


Критерий Дарбина-Уотсона применяется в ситуации, когда регрессионные остатки связаны **автокорреляционной зависимостью 1-го порядка**:

$$ \varepsilon_i = \rho \varepsilon_{i-1} + \delta_i, \quad i=\overline{2,n}$$

где $\rho$ - некоторое число ($|\rho|<1$), а случайные величины $\delta_i$ удовлетворяют требованиям, предъявляемым к регрессионным остаткам классической модели (т.е. равенство нулю среднего значения, постоянство дисперсии и некоррелированность между собой):

$$E(\delta_i)=0$$
$$ E(\delta_i\delta_j)=
\begin{cases}
{\sigma_0}^2, & \quad i=j \\
0, & \quad i \neq j
\end{cases} $$

Проверяется **нулевая гипотеза** об <u>отсутствии автокорреляции</u>:

$\begin{aligned}
\qquad H_0: \rho = 0
\end{aligned}$

**Альтернативной гипотезой** может быть:
* существование отрицательной автокорреляции (левосторонняя критическая область):

$\begin{aligned}
\qquad H_1: \rho < 0
\end{aligned}$  

* существование положительной автокорреляции (правосторонняя критическая область):

$\begin{aligned}
\qquad H_1: \rho > 0
\end{aligned}$  

* существование автокорреляции вообще (двусторонняя критическая область):

$\begin{aligned}
\qquad H_1: \rho \neq 0
\end{aligned}$

__Расчетное значение__ статистики $DW$ критерия Дарбина-Уотсона имеет вид:

$$ DW = \frac {\displaystyle\sum_{i=2}^{n} (\varepsilon_i - \varepsilon_{i-1})^2} {{\displaystyle\sum_{i=1}^{n} {\varepsilon_i}^2}} $$

где $\varepsilon_i = y_i - \widehat{y}_i$ - остатки (невязки) регрессионной модели.

По таблицам в зависимости от уровня значимости $\alpha$, числа регрессоров модели $k$ и объема выборки $n$ (от 15 до 100) определяются __критические значения статистики Дарбина-Уотсона__: <u>нижний</u> $d_L$ и <u>верхний</u> $d_U$ предел.

__Правила принятия решения по критерию Дарбина-Уотсона__

Критические значения образуют <u>пять областей</u> различных статистических решений

|Значение $DW$             |Гипотеза $H_0$                         |Вывод                            |
|:-------------------------------:|:--------------------------------------------:|:-------------------------------:|
|$0 \leq DW< d_L$         |отвергается|есть положительная автокорреляция|
|$d_L \leq DW \leq d_U$    |                                              |неопределенность                 |
|$d_U < DW < 4-d_U$        |не отвергается                             |автокорреляция отсутствует       |
|$4-d_U \leq DW \leq 4-d_L$|                                              |неопределенность                 |
|$4-d_L < DW \leq 4$       |отвергается|есть отрицательная автокорреляция|

Между статистикой критерия $DW$ и коэффициентом автокорреляции $\rho$ существует приближенное соотношение:

$$ DW \approx 2 \cdot (1-\rho) $$

Существуют и другие критерии для проверки автокорреляции (тест Бройша-Годфри, Льюнга-Бокса и пр.).

</div>

Рассмотрим регрессию

$$
	\text{наберите уравнение регрессии}
$$

In [ ]:
# загрузим данные
df = pd.read_csv('Labour.csv')

## Спецификация и подгонка

In [ ]:
# спецификация модели через формулу
mod = smf.ols(formula='your specification', data=df)
# подгонка модели с ковариационной матрицей по умолчанию (неробастной)
res = mod.fit()

## DW-тест

Тестируем модель на серийную корреляцию 1-го порядка

In [ ]:
# DW-статистика
sms.durbin_watson(res.resid)

## Вывод

ваш вывод

## LM/BG-тест

Тестируем модель на серийную корреляцию 1-го порядка 

In [ ]:
# LM-тест на автокорреляцию 1-го порядка
lm, lmpval, fval, fpval = sms.acorr_breusch_godfrey(res, nlags=1)
lm, lmpval

## Критическое значение $\chi^2$-распределения

In [ ]:
# Задаём уровень значимости
sign_level = 0.05
# Критическое значение распределения chi2
chi2.ppf(q=1-sign_level, df=mod.exog.shape[1])

## Вывод

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

<span style="color: purple"><i><b>Тест указывает на серийную корреляцию </i></b></span> (отвергаем $H_0$) т.к. $LM>\chi^2_{cr}$ или $P<\alpha$.

</div>

ваш вывод

## Робастный t-тест (HAC s.e.)

Используем HAC-оценку ковариационной матрицы

In [ ]:
# спецификация модели через формулу
mod = smf.ols(formula='sleep~1+totwrk+age+male+south+smsa+yngkid+marr+union', data=df)
# подгонка модели с робастной HC3-оценкой ковариационной матрицы
res_hac = mod.fit(cov_type='HAC')